In [1]:
# Licensing Information:  You are free to use or extend this project for
# educational purposes provided that (1) you do not distribute or publish
# solutions, (2) you retain this notice, and (3) you provide clear
# attribution to The Georgia Institute of Technology, including a link to https://aritter.github.io/CS-7650/

# Attribution Information: This assignment was developed at The Georgia Institute of Technology
# by Alan Ritter (alan.ritter@cc.gatech.edu)

# Project #2: Named Entity Recognition

In this assignment, you will implement a bidirectional LSTM-CNN-CRF for sequence labeling, following [this paper by Xuezhe Ma and Ed Hovy](https://www.aclweb.org/anthology/P16-1101.pdf), on the CoNLL named entity recognition dataset.  Before starting the assignment, we recommend reading the Ma and Hovy paper.

First, let's import some libraries and make sure the runtime has access to a GPU.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

print(f'GPU available: {torch.cuda.is_available()}')

Wed Mar 17 22:05:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Download the Data

Run the following code to download the English part of the CoNLL 2003 dataset, the evaluation script and pre-filtered GloVe embeddings we are providing for this data.

In [3]:
#CoNLL 2003 data
!wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.train
!wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.testa
!wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.testb
!cat eng.train | awk '{print $1 "\t" $4}' > train
!cat eng.testa | awk '{print $1 "\t" $4}' > dev
!cat eng.testb | awk '{print $1 "\t" $4}' > test

#Evaluation Script
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl

#Pre-filtered GloVe embeddings
!wget https://raw.githubusercontent.com/aritter/aritter.github.io/master/files/glove.840B.300d.conll_filtered.txt

--2021-03-17 22:05:58--  https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.train
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3283420 (3.1M) [text/plain]
Saving to: ‘eng.train’

eng.train           100%[===================>]   3.13M  --.-KB/s    in 0.1s    

2021-03-17 22:05:59 (27.4 MB/s) - ‘eng.train’ saved [3283420/3283420]

--2021-03-17 22:05:59--  https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.testa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response.

## CoNLL Data Format

Run the following cell to see a sample of the data in CoNLL format.  As you can see, each line in the file represents a word and its labeled named entity tag in BIO format.  A blank line is used to seperate sentences.

In [4]:
!head -n 20 train

-DOCSTART-	O
	
EU	I-ORG
rejects	O
German	I-MISC
call	O
to	O
boycott	O
British	I-MISC
lamb	O
.	O
	
Peter	I-PER
Blackburn	I-PER
	
BRUSSELS	I-LOC
1996-08-22	O
	
The	O
European	I-ORG


## Reading in the Data

Below we proivide a bit of code to read in data in the CoNLL format.  This also reads in the filtered GloVe embeddings, to save you some effort - we will discuss this more later.

In [5]:
import re

#Read in the training data.
def read_conll_format(filename):
  (words, tags, currentSent, currentTags) = ([],[],['-START-'],['START'])
  for line in open(filename).readlines():
    line = line.strip()
    #print(line)
    if line == "":
      currentSent.append('-END-')
      currentTags.append('END')
      words.append(currentSent)
      tags.append(currentTags)
      (currentSent, currentTags) = (['-START-'], ['START'])
    else:
      (word, tag) = line.split()
      currentSent.append(word)
      currentTags.append(tag)
  return (words, tags)

#Read GloVe embeddings.
def read_GloVe(filename):
  embeddings = {}
  for line in open(filename).readlines():
    #print(line)
    fields = line.strip().split(" ")
    word = fields[0]
    embeddings[word] = [float(x) for x in fields[1:]]
  return embeddings

GloVe = read_GloVe("glove.840B.300d.conll_filtered.txt")

(sentences_train, tags_train) = read_conll_format("train")
(sentences_dev, tags_dev)     = read_conll_format("dev")

def sentences2char(sentences):
  return [[['start'] + [c for c in w] + ['end'] for w in l] for l in sentences]

sentencesChar = sentences2char(sentences_train)

print(sentences_train[2])
print(tags_train[2])

print(GloVe['the'])

['-START-', 'Peter', 'Blackburn', '-END-']
['START', 'I-PER', 'I-PER', 'END']
[0.27204, -0.06203, -0.1884, 0.023225, -0.018158, 0.0067192, -0.13877, 0.17708, 0.17709, 2.5882, -0.35179, -0.17312, 0.43285, -0.10708, 0.15006, -0.19982, -0.19093, 1.1871, -0.16207, -0.23538, 0.003664, -0.19156, -0.085662, 0.039199, -0.066449, -0.04209, -0.19122, 0.011679, -0.37138, 0.21886, 0.0011423, 0.4319, -0.14205, 0.38059, 0.30654, 0.020167, -0.18316, -0.0065186, -0.0080549, -0.12063, 0.027507, 0.29839, -0.22896, -0.22882, 0.14671, -0.076301, -0.1268, -0.0066651, -0.052795, 0.14258, 0.1561, 0.05551, -0.16149, 0.09629, -0.076533, -0.049971, -0.010195, -0.047641, -0.16679, -0.2394, 0.0050141, -0.049175, 0.013338, 0.41923, -0.10104, 0.015111, -0.077706, -0.13471, 0.119, 0.10802, 0.21061, -0.051904, 0.18527, 0.17856, 0.041293, -0.014385, -0.082567, -0.035483, -0.076173, -0.045367, 0.089281, 0.33672, -0.22099, -0.0067275, 0.23983, -0.23147, -0.88592, 0.091297, -0.012123, 0.013233, -0.25799, -0.02972, 0.0167

## Mapping Tokens to Indices

As in the last project, we will need to convert words in the dataset to numeric indices, so they can be presented as input to a neural network.  Code to handle this for you with sample usage is provided below.

In [6]:
#Create mappings between tokens and indices.

from collections import Counter
import random

#Will need this later to remove 50% of words that only appear once in the training data from the vocabulary (and don't have GloVe embeddings).
wordCounts = Counter([w for l in sentences_train for w in l])
charCounts = Counter([c for l in sentences_train for w in l for c in w])
singletons = set([w for (w,c) in wordCounts.items() if c == 1 and not w in GloVe.keys()])
charSingletons = set([w for (w,c) in charCounts.items() if c == 1])

#Build dictionaries to map from words, characters to indices and vice versa.
#Save first two words in the vocabulary for padding and "UNK" token.
word2i = {w:i+2 for i,w in enumerate(set([w for l in sentences_train for w in l] + list(GloVe.keys())))}
char2i = {w:i+2 for i,w in enumerate(set([c for l in sentencesChar for w in l for c in w]))}
i2word = {i:w for w,i in word2i.items()}
i2char  = {i:w for w,i in char2i.items()}

#Tag dictionaries.
tag2i = {w:i for i,w in enumerate(set([t for l in tags_train for t in l]))}
i2tag = {i:t for t,i in tag2i.items()}

#When training, randomly replace singletons with UNK tokens sometimes to simulate situation at test time.
def getDictionaryRandomUnk(w, dictionary, train=False):
  if train and (w in singletons and random.random() > 0.5):
    return 1
  else:
    return dictionary.get(w, 1)

#Map a list of sentences from words to indices.
def sentences2indices(words, dictionary, train=False):
  #1.0 => UNK
  return [[getDictionaryRandomUnk(w,dictionary, train=train) for w in l] for l in words]

#Map a list of sentences containing to indices (character indices)
def sentences2indicesChar(chars, dictionary):
  #1.0 => UNK
  return [[[dictionary.get(c,1) for c in w] for w in l] for l in chars]

#Indices
X       = sentences2indices(sentences_train, word2i, train=True)
X_char  = sentences2indicesChar(sentencesChar, char2i)
Y       = sentences2indices(tags_train, tag2i)

print(i2word[253])

#Print out some examples of what the dev inputs will look like
for i in range(10):
  print(" ".join([i2word.get(w,'UNK') for w in X[i]]))

Ironi
-START- -DOCSTART- -END-
-START- EU rejects German call to boycott British lamb . -END-
-START- Peter Blackburn -END-
-START- BRUSSELS 1996-08-22 -END-
-START- The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep . -END-
-START- Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer . -END-
-START- " We do n't support any such recommendation because we do n't see any grounds for it , " the Commission 's chief spokesman Nikolaus van der Pas told a news briefing . -END-
-START- He said further scientific study was required and if it was found that action was needed it should be taken by the European Union . -END-
-START- He said a proposal last month by EU Farm Commissioner Franz Fischler to ban s

## Padding and Batching

In this assignment, you should train your models using minibatched SGD, rather than using a batch size of 1 as we did in the previous project.  When presenting multiple sentences to the network at the same time, we will need to pad them to be of the same length.

Below we provide some code to prepare batches of data to present to the network.  

**Side Note:** PyTorch includes utilities in [`torch.utils.data`](https://pytorch.org/docs/stable/data.html) to help with padding, batching, shuffling and some other things, but for this assignment we will do everything from scratch to help you see exactly how this works.

In [7]:
#Pad inputs to max sequence length (for batching)
def prepare_input(X_list):
  X_padded = torch.nn.utils.rnn.pad_sequence([torch.as_tensor(l) for l in X_list], batch_first=True).type(torch.LongTensor)
  X_mask   = torch.nn.utils.rnn.pad_sequence([torch.as_tensor([1.0] * len(l)) for l in X_list], batch_first=True).type(torch.FloatTensor)
  return (X_padded, X_mask)

#Maximum word length (for character representations)
MAX_CLEN=32

def prepare_input_char(X_list):
  MAX_SLEN = max([len(l) for l in X_list])
  X_padded  = [l + [[]]*(MAX_SLEN-len(l))  for l in X_list]
  X_padded  = [[w[0:MAX_CLEN] for w in l] for l in X_padded]
  X_padded  = [[w + [1]*(MAX_CLEN-len(w)) for w in l] for l in X_padded]
  #X_mask    = [[1]*len(l) + [0]*(MAX_SLEN-len(l)) for l in X_list]
  return torch.as_tensor(X_padded).type(torch.LongTensor)

#Pad outputs using one-hot encoding
def prepare_output_onehot(Y_list, NUM_TAGS=max(tag2i.values())+1):
  Y_onehot = [torch.zeros(len(l), NUM_TAGS) for l in Y_list]
  for i in range(len(Y_list)):
   for j in range(len(Y_list[i])):
    Y_onehot[i][j,Y_list[i][j]] = 1.0
  Y_padded = torch.nn.utils.rnn.pad_sequence(Y_onehot, batch_first=True).type(torch.FloatTensor)
  return Y_padded

print("max slen:", max([len(x) for x in X_char]))  #Max sequence length in the training data is 39.

(X_padded, X_mask) = prepare_input(X)
X_padded_char = prepare_input_char(X_char)
Y_onehot = prepare_output_onehot(Y)

print("X_padded:", X_padded.shape)
print("X_padded_char:", X_padded_char.shape)
print("Y_onehot:", Y_onehot.shape)

max slen: 115
X_padded: torch.Size([14987, 115])
X_padded_char: torch.Size([14987, 115, 32])
Y_onehot: torch.Size([14987, 115, 10])


## **Your code starts here:** Basic LSTM Tagger (10 points)

OK, now you should have everything you need to get started.

Recall that your goal is to to implement the BiLSTM-CNN-CRF, as described in [(Ma and Hovy, 2016)](https://www.aclweb.org/anthology/P16-1101.pdf).  This is a relatively complex network with various components.  Below we provide starter code to break down your implementation into increasingly complex versions of the final model, starting with a Basic LSTM tagger.  This way you can be confident that each part is working correctly before incrementally increasing the complexity of your implementation.  This is generally a good approach to take when implementing complex models, since buggy PyTorch code is often partially working, but produces worse results than a correct implementation, so it's hard to know whether added complexities are helping or hurting.  Also, if you aren't able to match published results it's hard to know which component of your model has the problem (or even whether or not it is a problem in the published result!)

Fill in the functions marked as `TODO` in the code block below.  If everything is working correctly, you should be able to achieve an F1 score of 0.87 on the dev set and 0.83 on the test set (with GloVe embeddings).

In [8]:
len(char2i)

86

In [9]:
GloVe['a']

[0.043798,
 0.024779,
 -0.20937,
 0.49745,
 0.36019,
 -0.37503,
 -0.052078,
 -0.60555,
 0.036744,
 2.2085,
 -0.23389,
 -0.06836,
 -0.22355,
 -0.053989,
 -0.15198,
 -0.17319,
 0.053355,
 1.6485,
 -0.047991,
 -0.085311,
 -0.15712,
 -0.64425,
 -0.39819,
 0.278,
 0.15364,
 0.031678,
 0.055414,
 0.015939,
 0.31851,
 -0.058979,
 0.038584,
 0.1077,
 0.1041,
 -0.077346,
 0.37396,
 -0.21482,
 0.3832,
 -0.27737,
 -0.18352,
 -0.83838,
 0.34124,
 0.58164,
 0.18543,
 -0.31028,
 0.17666,
 -0.069421,
 -0.34422,
 -0.13665,
 -0.10823,
 0.23637,
 -0.32923,
 0.61348,
 0.1972,
 0.087123,
 0.10785,
 0.3073,
 0.13757,
 0.30809,
 0.24331,
 -0.29422,
 -0.0098214,
 0.55675,
 -0.04888,
 0.099468,
 0.30543,
 -0.37597,
 -0.19525,
 0.046246,
 -0.036675,
 0.34023,
 0.14905,
 0.0978,
 -0.26664,
 0.056834,
 -0.043201,
 -0.23338,
 0.13111,
 -0.35742,
 -0.3607,
 0.30997,
 -0.19727,
 -0.1432,
 -0.16747,
 0.00042435,
 -0.1512,
 0.067562,
 -0.38644,
 0.025349,
 0.24918,
 -0.23955,
 -0.15615,
 0.49868,
 0.0082758,
 -0.1912

In [10]:
import numpy as np
class BasicLSTMtagger(nn.Module):
    def __init__(self, DIM_EMB=10, DIM_CHAR_EMB=32, DIM_HID=10):
        super(BasicLSTMtagger, self).__init__()
        NUM_TAGS = max(tag2i.values())+1

        (self.DIM_EMB, self.NUM_TAGS) = (DIM_EMB, NUM_TAGS)
        #TODO: initialize parameters - embedding layer, nn.LSTM, nn.Linear and nn.LogSoftmax
        #print(NUM_TAGS)
        vocab_size = len(word2i) +3

        self.init_glove(GloVe)
        emb_matrix = np.zeros((vocab_size, DIM_EMB))
        emb_vec_len = 300
        for word, idx in word2i.items():
          if word in GloVe:
            emb_matrix[idx,:] = GloVe[word]
          else:
            emb_matrix[idx,:] = [0]*emb_vec_len
          

        #self.word_embedding_layer.weight.data[word2i[word]] = torch.as_tensor(GloVe[word])

        #self.glove_emb = nn.Embedding(vocab_size, DIM_EMB)


        self.embedding = nn.Embedding( vocab_size, DIM_EMB)
        # for word in word2i.items():
          # if word in self.glove:
#        self.embedding.weight = torch.as_tensor(GloVe)
        self.embedding.weight=nn.Parameter(torch.tensor(emb_matrix,dtype=torch.float32))


        self.lstm = nn.LSTM(input_size=DIM_EMB, hidden_size=DIM_HID,num_layers=1, batch_first=True, bidirectional=True)
        self.linear = nn.Linear(2*DIM_HID, NUM_TAGS)
        #self.linear = nn.Linear(DIM_HID, NUM_TAGS)
        self.softmax = nn.LogSoftmax(2)

    def forward(self, X, train=False):
        #TODO: Implement the forward computation.
        #return torch.randn((X.shape[0], X.shape[1], self.NUM_TAGS))  #Random baseline.
        #print(X.shape)
        #X = X.cuda()
        # if X in self.glove:
        #   emb=self.glove_emb
        # else:
        emb = self.embedding(X)
        #print(emb.shape)
        lstm_out, _ = self.lstm(emb)
        lin = self.linear(lstm_out)
        sm = self.softmax(lin)
        return sm


    def init_glove(self, GloVe):
      #TODO: initialize word embeddings using GloVe (you can skip this part in your first version, if you want, see instructions below).
      self.glove = GloVe

    def inference(self, sentences):
      X       = prepare_input(sentences2indices(sentences, word2i))[0].cuda()
      pred = self.forward(X).argmax(dim=2)
      return [[i2tag[pred[i,j].item()] for j in range(len(sentences[i]))] for i in range(len(sentences))]

    def print_predictions(self, words, tags):
      Y_pred = self.inference(words)
      for i in range(len(words)):
        print("----------------------------")
        print(" ".join([f"{words[i][j]}/{Y_pred[i][j]}/{tags[i][j]}" for j in range(len(words[i]))]))
        print("Predicted:\t", Y_pred[i])
        print("Gold:\t\t", tags[i])

    def write_predictions(self, sentences, outFile):
      fOut = open(outFile, 'w')
      for s in sentences:
        y = self.inference([s])[0]
        #print("\n".join(y[1:len(y)-1]))
        fOut.write("\n".join(y[1:len(y)-1]))  #Skip start and end tokens
        fOut.write("\n\n")

#The following code will initialize a model and test that your forward computation runs without errors.
lstm        = BasicLSTMtagger(DIM_HID=7, DIM_EMB=300)
lstm_output = lstm.forward(prepare_input(X[0:5])[0])
Y_onehot    = prepare_output_onehot(Y[0:5])

#Check the shape of the lstm_output and one-hot label tensors.
print("lstm output shape:", lstm_output.shape)
print("Y onehot shape:", Y_onehot.shape)

lstm output shape: torch.Size([5, 32, 10])
Y onehot shape: torch.Size([5, 32, 10])


In [11]:

#Read in the data

(sentences_dev, tags_dev)     = read_conll_format('dev')
(sentences_train, tags_train) = read_conll_format('train')
(sentences_test, tags_test)   = read_conll_format('test')

# Train your Model (10 points)

Next, implement the function below to train your basic BiLSTM tagger.  See [torch.nn.lstm](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html).  Make sure to save your predictions on the test set (`test_pred_lstm.txt`) for submission to GradeScope.

In [12]:
#!CUDA_LAUNCH_BLOCKING=1

In [13]:
#Training

from random import sample
import tqdm
import os
import subprocess
import random

def shuffle_sentences(sentences, tags):
  shuffled_sentences = []
  shuffled_tags      = []
  indices = list(range(len(sentences)))
  random.shuffle(indices)
  for i in indices:
    #print(len(sentences[i]), len(tags[i]))
    shuffled_sentences.append(sentences[i])
    shuffled_tags.append(tags[i])
  return (shuffled_sentences, shuffled_tags)

nEpochs = 10

def train_basic_lstm(sentences, tags, lstm):
 # print(lstm.parameters())
  
  #optimizer = optim.Adadelta(lstm.parameters(), lr=0.015)

  batchSize =10

  for epoch in range(nEpochs):
    totalLoss = 0.0
    #.015
    nt = .15 / (1 +0.05*epoch)
    optimizer = optim.Adadelta(lstm.parameters(), lr=nt)


    (sentences_shuffled, tags_shuffled) = shuffle_sentences(sentences, tags)
    for batch in tqdm.notebook.tqdm(range(0, len(sentences), batchSize), leave=False):
      lstm.zero_grad()

      input = sentences2indices(sentences_shuffled[batch:batchSize+batch], word2i, train=True)
      #print(input)
      lstm = lstm.cuda()
      #lstm_input = prepare_input(w)
      lstm_input0, lstm_input1 = prepare_input(input)

      lstm_input0, lstm_input1 =  lstm_input0.cuda(), lstm_input1.cuda()

      scores = lstm.forward(lstm_input0)
      #print(scores)

      input_tags = sentences2indices(tags_shuffled[batch:batchSize+batch], tag2i, train=True)
      #print(input_tags)
      prep_input_tags = prepare_output_onehot(input_tags)
     # prep_input_tags = prepare_output_onehot(input_tags)
      #print(prep_input_tags)

      #loss = torch.sum(torch.sum(torch.neg(scores.cuda()) * prep_input_tags.cuda() )* lstm_input1.cuda())
      loss = torch.sum(torch.neg(scores.cuda()) * prep_input_tags.cuda())
      #print(scores.size())
      #input_tags_as_tensor - 
      #print(torch.FloatTensor(input_tags).size())      
      #loss = loss_function(scores.cuda(),torch.FloatTensor(flat_list).cuda())

      #print(loss)
      loss.backward()
      #loss = 0

      torch.nn.utils.clip_grad_norm_(lstm.parameters(), 5)

      totalLoss += loss
      optimizer.step()
      #loss=0

    print(f"loss on epoch {epoch} = {totalLoss}")
    lstm.write_predictions(sentences_dev, 'dev_pred')   #Performance on dev set
    print('conlleval:')
    print(subprocess.Popen('paste dev dev_pred | perl conlleval.pl -d "\t"', shell=True, stdout=subprocess.PIPE,stderr=subprocess.STDOUT).communicate()[0].decode('UTF-8'))

    if epoch % 10 == 0:
      s = sample(range(len(sentences_dev)), 5)
      lstm.print_predictions([sentences_dev[i] for i in s], [tags_dev[i] for i in s])

lstm = BasicLSTMtagger(DIM_HID=500, DIM_EMB=300).cuda()

train_basic_lstm(sentences_train, tags_train, lstm)

loss on epoch 0 = 39349.8515625
conlleval:
processed 51578 tokens with 5942 phrases; found: 6020 phrases; correct: 5035.
accuracy:  97.42%; precision:  83.64%; recall:  84.74%; FB1:  84.18
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  5
              LOC: precision:  90.48%; recall:  87.97%; FB1:  89.21  1786
             MISC: precision:  82.68%; recall:  73.54%; FB1:  77.84  820
              ORG: precision:  71.73%; recall:  78.15%; FB1:  74.80  1461
              PER: precision:  86.91%; recall:  91.91%; FB1:  89.34  1948

----------------------------
-START-/START/START "/O/O This/O/O is/O/O really/O/O positive/O/O news/O/O for/O/O Pirelli/I-ORG/I-ORG ,/O/O and/O/O I/O/O expect/O/O that/O/O it/O/O will/O/O produce/O/O one/O/O of/O/O the/O/O best/O/O half-year/O/O results/O/O in/O/O late/O/O September/O/O compared/O/O to/O/O other/O/O industrial/O/O Italian/I-MISC/I-MISC companies/O/O ,/O/O "/O/O said/O/O analyst/O/O Paula/I-PER/I-PER Buratti/I-PER/I-PER at/O

loss on epoch 1 = 14879.0703125
conlleval:
processed 51578 tokens with 5942 phrases; found: 5956 phrases; correct: 5204.
accuracy:  98.02%; precision:  87.37%; recall:  87.58%; FB1:  87.48
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  3
              LOC: precision:  92.17%; recall:  91.62%; FB1:  91.89  1826
             MISC: precision:  81.61%; recall:  79.39%; FB1:  80.48  897
              ORG: precision:  78.51%; recall:  81.21%; FB1:  79.84  1387
              PER: precision:  92.24%; recall:  92.29%; FB1:  92.27  1843



loss on epoch 2 = 10771.8505859375
conlleval:
processed 51578 tokens with 5942 phrases; found: 5998 phrases; correct: 5349.
accuracy:  98.34%; precision:  89.18%; recall:  90.02%; FB1:  89.60
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  2
              LOC: precision:  91.91%; recall:  95.81%; FB1:  93.82  1915
             MISC: precision:  84.17%; recall:  81.89%; FB1:  83.01  897
              ORG: precision:  84.24%; recall:  83.30%; FB1:  83.76  1326
              PER: precision:  92.41%; recall:  93.21%; FB1:  92.81  1858



loss on epoch 3 = 8390.638671875
conlleval:
processed 51578 tokens with 5942 phrases; found: 6033 phrases; correct: 5389.
accuracy:  98.44%; precision:  89.33%; recall:  90.69%; FB1:  90.00
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  2
              LOC: precision:  92.63%; recall:  95.10%; FB1:  93.85  1886
             MISC: precision:  83.74%; recall:  82.10%; FB1:  82.91  904
              ORG: precision:  84.55%; recall:  84.49%; FB1:  84.52  1340
              PER: precision:  92.16%; recall:  95.11%; FB1:  93.61  1901



loss on epoch 4 = 6740.5615234375
conlleval:
processed 51578 tokens with 5942 phrases; found: 5986 phrases; correct: 5404.
accuracy:  98.52%; precision:  90.28%; recall:  90.95%; FB1:  90.61
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  1
              LOC: precision:  93.92%; recall:  95.05%; FB1:  94.48  1859
             MISC: precision:  84.57%; recall:  83.84%; FB1:  84.20  914
              ORG: precision:  85.48%; recall:  85.16%; FB1:  85.32  1336
              PER: precision:  92.91%; recall:  94.63%; FB1:  93.76  1876



loss on epoch 5 = 5373.53857421875
conlleval:
processed 51578 tokens with 5942 phrases; found: 6012 phrases; correct: 5440.
accuracy:  98.59%; precision:  90.49%; recall:  91.55%; FB1:  91.02
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  1
              LOC: precision:  94.05%; recall:  95.43%; FB1:  94.73  1864
             MISC: precision:  85.54%; recall:  82.75%; FB1:  84.12  892
              ORG: precision:  85.63%; recall:  85.76%; FB1:  85.69  1343
              PER: precision:  92.78%; recall:  96.31%; FB1:  94.51  1912



loss on epoch 6 = 4223.02685546875
conlleval:
processed 51578 tokens with 5942 phrases; found: 5971 phrases; correct: 5415.
accuracy:  98.55%; precision:  90.69%; recall:  91.13%; FB1:  90.91
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  1
              LOC: precision:  93.95%; recall:  95.48%; FB1:  94.71  1867
             MISC: precision:  85.28%; recall:  82.97%; FB1:  84.11  897
              ORG: precision:  86.16%; recall:  85.91%; FB1:  86.03  1337
              PER: precision:  93.31%; recall:  94.68%; FB1:  93.99  1869



loss on epoch 7 = 3368.73193359375
conlleval:
processed 51578 tokens with 5942 phrases; found: 5974 phrases; correct: 5385.
accuracy:  98.45%; precision:  90.14%; recall:  90.63%; FB1:  90.38
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  1
              LOC: precision:  94.65%; recall:  94.45%; FB1:  94.55  1833
             MISC: precision:  85.54%; recall:  82.10%; FB1:  83.79  885
              ORG: precision:  83.95%; recall:  84.27%; FB1:  84.11  1346
              PER: precision:  92.35%; recall:  95.71%; FB1:  94.00  1909



loss on epoch 8 = 2550.041748046875
conlleval:
processed 51578 tokens with 5942 phrases; found: 6000 phrases; correct: 5401.
accuracy:  98.51%; precision:  90.02%; recall:  90.90%; FB1:  90.45
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  1
              LOC: precision:  94.55%; recall:  94.39%; FB1:  94.47  1834
             MISC: precision:  83.71%; recall:  84.16%; FB1:  83.94  927
              ORG: precision:  83.75%; recall:  85.68%; FB1:  84.70  1372
              PER: precision:  93.35%; recall:  94.57%; FB1:  93.96  1866



loss on epoch 9 = 1915.3970947265625
conlleval:
processed 51578 tokens with 5942 phrases; found: 6024 phrases; correct: 5411.
accuracy:  98.52%; precision:  89.82%; recall:  91.06%; FB1:  90.44
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  2
              LOC: precision:  93.73%; recall:  95.21%; FB1:  94.46  1866
             MISC: precision:  84.35%; recall:  82.43%; FB1:  83.38  901
              ORG: precision:  84.34%; recall:  85.53%; FB1:  84.93  1360
              PER: precision:  92.61%; recall:  95.28%; FB1:  93.93  1895



In [14]:
#Evaluation on test data
lstm.write_predictions(sentences_test, 'test_pred_lstm.txt')
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
!paste test test_pred_lstm.txt | perl conlleval.pl -d "\t"

--2021-03-17 22:09:06--  https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12754 (12K) [text/plain]
Saving to: ‘conlleval.pl.1’

conlleval.pl.1      100%[===================>]  12.46K  --.-KB/s    in 0s      

2021-03-17 22:09:06 (139 MB/s) - ‘conlleval.pl.1’ saved [12754/12754]

processed 46666 tokens with 5648 phrases; found: 5748 phrases; correct: 4928.
accuracy:  97.47%; precision:  85.73%; recall:  87.25%; FB1:  86.49
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  1
              LOC: precision:  88.15%; recall:  91.43%; FB1:  89.76  1730
             MISC: precision:  72.39%; recall:  75.07%; FB1:  73.71  728
              ORG: precision:

## Initialization with GloVe Embeddings (5 points)

If you haven't already, implement the `init_glove()` method in `BasicLSTMtagger` above.

Rather than initializing word embeddings randomly, it is common to use learned word embeddings (GloVe or Word2Vec), as discussed in lecture.  To make this simpler, we have already pre-filtered [GloVe](https://nlp.stanford.edu/projects/glove/) embeddings to only contain words in the vocabulary of the CoNLL NER dataset, and loaded them into a dictionary (`GloVe`) at the beginning of this notebook.



## Character Embeddings (10 points)

Now that you have your basic LSTM tagger working, the next step is to add a convolutional network that computes word embeddings from character representations of words.  See Figure 2 and Figure 3 in the [Ma and Hovy](https://www.aclweb.org/anthology/P16-1101.pdf) paper.  We have provided code in `sentences2input_tensors` to convert sentences into lists of word and character indices.  See also [nn.Conv1d](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html) and [MaxPool1d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool1d.html).

Make sure to save your predictions on the test set, for submission to GradeScope.  You should be able to achieve 90 F1 / 85 F1 on the dev/test sets.

In [15]:
#For F.max_pool1d()
import torch.nn.functional as F

#class CharLSTMtagger(nn.Module):
class CharLSTMtagger(BasicLSTMtagger):
    def __init__(self, DIM_EMB=10, DIM_CHAR_EMB=30, DIM_HID=10):
        super(CharLSTMtagger, self).__init__(DIM_EMB=DIM_EMB, DIM_HID=DIM_HID)
        NUM_TAGS = max(tag2i.values())+1

        (self.DIM_EMB, self.NUM_TAGS) = (DIM_EMB, NUM_TAGS)
        self.DIM_CHAR_EMB = DIM_CHAR_EMB
        #TODO: Initialize parameters.
        #char_emb_matrix = np.zeros((vocab_size, DIM_EMB))


        self.char_embedding = nn.Embedding(88,DIM_CHAR_EMB)
        #self.char_embedding.weight=nn.Parameter(torch.tensor(emb_matrix,dtype=torch.float32))

        self.conv = nn.Conv1d(DIM_CHAR_EMB,DIM_CHAR_EMB,3)
        self.pool = nn.MaxPool1d(30)
        self.lstm = nn.LSTM(input_size=DIM_EMB+DIM_CHAR_EMB, hidden_size=DIM_HID,num_layers=1, batch_first=True, bidirectional=True)

    def forward(self, X, X_char, train=False):
        #TODO: Implement the forward computation.
        #return torch.randn((X.shape[0], X.shape[1], self.NUM_TAGS))  #Random baseline.
        #print(X)
        #X, X_max, X_char = self.sentences2input_tensors(X)
        #char emb
        
        char_emb = self.char_embedding(X_char)
        shape = char_emb.size()
        #print("shape after char embedding")
        #print(shape)

        shape_perm = char_emb.size()
        #char_emb_reshape = char_emb.view(tuple((-1, *shape[2:])))
        #print(shape_perm)
        #char_emb_reshape = char_emb.reshape(char_emb, 160, 30, 32)
        char_emb= char_emb.permute(0,1,3,2)
        #print("size after permute")
        #print(char_emb.size())

        #char_emb_reshape = char_emb.view(tuple((-1, *shape[2:])))
        char_emb_flat = torch.flatten(char_emb, start_dim=0,end_dim=1)
       # print("size after flat")
       # print(char_emb_flat.size())
        conv = self.conv(char_emb_flat)
        #print(conv.size())
        #print("shape after conv")
        #print(conv.size())
        #max pool
        #pool= F.max_pool1d(conv, 30)
        #pool = self.pool(conv)
        pool= torch.max(conv,2)[0]
        # # concat to word emb
        #print(pool)
        # #reshape pool then concat to emb
       # print("shape after pool")
        psize= pool.size()
        #print(psize)
        #print("unflat")
        #unflat_pool = pool.unflatten(0, (5,32))
        #print(unflat_pool.size())
        unflat_pool = pool.reshape(X.shape[0], X.shape[1] , self.DIM_CHAR_EMB)
        # print("size after unflat")
        # print(unflat_pool.size())
       # pool_reshape = torch.flatten(unflat_pool, start_dim=2,end_dim=3)
        # print("pool reshape size")
        # print(pool_reshape.size())
        emb = self.embedding(X)
        #print(emb)
        #print("word embedding size")
        #print(emb.size())
        #print(emb.shape)
        concat = torch.cat((emb, unflat_pool), dim =2)

        lstm_out, _ = self.lstm(concat)
        lin = self.linear(lstm_out)
        sm = self.softmax(lin)
        return sm
    def sentences2input_tensors(self, sentences):
      (X, X_mask)   = prepare_input(sentences2indices(sentences, word2i))
      X_char        = prepare_input_char(sentences2indicesChar(sentences, char2i))
      return (X, X_mask, X_char)

    def inference(self, sentences):
      (X, X_mask, X_char) = self.sentences2input_tensors(sentences)
      pred = self.forward(X.cuda(), X_char.cuda()).argmax(dim=2)
      return [[i2tag[pred[i,j].item()] for j in range(len(sentences[i]))] for i in range(len(sentences))]

    def print_predictions(self, words, tags):
      Y_pred = self.inference(words)
      for i in range(len(words)):
        print("----------------------------")
        print(" ".join([f"{words[i][j]}/{Y_pred[i][j]}/{tags[i][j]}" for j in range(len(words[i]))]))
        print("Predicted:\t", Y_pred[i])
        print("Gold:\t\t", tags[i])

lstm        = CharLSTMtagger(DIM_HID=7, DIM_EMB=300)
lstm_output = lstm.forward(prepare_input(X[0:5])[0], prepare_input_char(X_char[0:5]))
Y_onehot    = prepare_output_onehot(Y[0:5])

print("lstm output shape:", lstm_output.shape)
print("Y onehot shape:", Y_onehot.shape)

lstm output shape: torch.Size([5, 32, 10])
Y onehot shape: torch.Size([5, 32, 10])


In [16]:
#Training LSTM w/ character embeddings.

nEpochs = 10

def train_char_lstm(sentences, tags, lstm):
  optimizer = optim.Adadelta(lstm.parameters(), lr=0.1)
  #TODO: initialize optimizer
  loss_fn = nn.CrossEntropyLoss()


  batchSize = 5

  for epoch in range(nEpochs):
    totalLoss = 0.0

    (sentences_shuffled, tags_shuffled) = shuffle_sentences(sentences, tags)
    for batch in tqdm.notebook.tqdm(range(0, len(sentences), batchSize), leave=False):
      lstm.zero_grad()


      (X, X_mask, X_char) = lstm.sentences2input_tensors(sentences_shuffled[batch:batchSize+batch])
    #  print("size of X")
     # print(X.size())
     # print("size of X_char")
     # print(X_char.size())
      X = X.cuda()
      X_mask = X_mask.cuda()
      X_char= X_char.cuda()
      lstm = lstm.cuda()

      scores = lstm.forward(X, X_char, train=True)
      #print(scores)
      #print("X_mask size")
      #print(X_mask.size())
      input_tags = sentences2indices(tags_shuffled[batch:batchSize+batch], tag2i, train=True)

      prep_input_tags = prepare_output_onehot(input_tags)
     # prep_input_tags = prepare_output_onehot(input_tags)
      #print(prep_input_tags)
      #print("onehot input tag size")
      #print(prep_input_tags.size())
      #loss = torch.sum(torch.sum(torch.neg(scores.cuda()) * prep_input_tags.cuda() )* lstm_input1.cuda())
#      loss = torch.sum(torch.neg(scores.cuda()) * prep_input_tags.cuda())
     # print("scores size")
     # print(scores.size())
     # target= torch.argmax(prep_input_tags.cuda(), dim=1)
     # print("target size")
     # print(target.size())
      #mult = torch.neg(scores.cuda()) * prep_input_tags.cuda()
      #loss = torch.sum(mult)
      #scores= scores.cuda()
      #target = target.cuda()
      #loss = loss_fn(scores.cuda(), target.cuda())
     # mult = scores*X_mask
     # print("mult size")
     # print(mult.size())
      loss = torch.sum(torch.neg(scores.cuda() ) *prep_input_tags.cuda())

      #print(scores.size())
      #input_tags_as_tensor - 
      #print(torch.FloatTensor(input_tags).size())      
      #loss = loss_function(scores.cuda(),torch.FloatTensor(flat_list).cuda())

      #print(loss)
      loss.backward()
      #loss = 0

      torch.nn.utils.clip_grad_norm_(lstm.parameters(), 5)

      totalLoss += loss
      optimizer.step()
      #loss=0

    print(f"loss on epoch {epoch} = {totalLoss}")
    lstm.write_predictions(sentences_dev, 'dev_pred')   #Performance on dev set
    print('conlleval:')
    print(subprocess.Popen('paste dev dev_pred | perl conlleval.pl -d "\t"', shell=True, stdout=subprocess.PIPE,stderr=subprocess.STDOUT).communicate()[0].decode('UTF-8'))

    if epoch % 10 == 0:
      s = sample(range(len(sentences_dev)), 5)
      lstm.print_predictions([sentences_dev[i] for i in s], [tags_dev[i] for i in s])

char_lstm = CharLSTMtagger(DIM_HID=500, DIM_EMB=300).cuda()
train_char_lstm(sentences_train, tags_train, char_lstm)

loss on epoch 0 = 33601.609375
conlleval:
processed 51578 tokens with 5942 phrases; found: 6136 phrases; correct: 5079.
accuracy:  97.44%; precision:  82.77%; recall:  85.48%; FB1:  84.10
              LOC: precision:  83.80%; recall:  93.20%; FB1:  88.25  2043
             MISC: precision:  79.54%; recall:  67.90%; FB1:  73.26  787
              ORG: precision:  72.13%; recall:  79.12%; FB1:  75.46  1471
              PER: precision:  91.55%; recall:  91.21%; FB1:  91.38  1835

----------------------------
-START-/START/START U.S./I-LOC/I-LOC President/O/O Bill/I-PER/I-PER Clinton/I-PER/I-PER has/O/O authorised/O/O the/O/O repositioning/O/O of/O/O U.S./I-LOC/I-LOC firepower/O/O in/O/O the/O/O Gulf/I-LOC/I-LOC region/O/O in/O/O response/O/O to/O/O the/O/O Iraqi/I-MISC/I-MISC attacks/O/O ./O/O -END-/END/END
Predicted:	 ['START', 'I-LOC', 'O', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'I-LOC', 'O', 'O', 'O', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'I-MISC', 'O', 'O', 'END']
Gold:		 ['START', 

loss on epoch 1 = 15709.107421875
conlleval:
processed 51578 tokens with 5942 phrases; found: 6064 phrases; correct: 5266.
accuracy:  98.01%; precision:  86.84%; recall:  88.62%; FB1:  87.72
              LOC: precision:  94.17%; recall:  91.45%; FB1:  92.79  1784
             MISC: precision:  82.67%; recall:  78.63%; FB1:  80.60  877
              ORG: precision:  74.29%; recall:  85.31%; FB1:  79.42  1540
              PER: precision:  92.16%; recall:  93.21%; FB1:  92.69  1863



loss on epoch 2 = 12179.23046875
conlleval:
processed 51578 tokens with 5942 phrases; found: 5998 phrases; correct: 5333.
accuracy:  98.25%; precision:  88.91%; recall:  89.75%; FB1:  89.33
              LOC: precision:  92.55%; recall:  93.96%; FB1:  93.25  1865
             MISC: precision:  85.31%; recall:  78.09%; FB1:  81.54  844
              ORG: precision:  79.48%; recall:  84.64%; FB1:  81.98  1428
              PER: precision:  94.14%; recall:  95.11%; FB1:  94.63  1861



loss on epoch 3 = 9939.4677734375
conlleval:
processed 51578 tokens with 5942 phrases; found: 6085 phrases; correct: 5371.
accuracy:  98.35%; precision:  88.27%; recall:  90.39%; FB1:  89.32
              LOC: precision:  88.38%; recall:  96.90%; FB1:  92.44  2014
             MISC: precision:  85.15%; recall:  79.61%; FB1:  82.29  862
              ORG: precision:  83.40%; recall:  83.15%; FB1:  83.27  1337
              PER: precision:  93.06%; recall:  94.57%; FB1:  93.81  1872



loss on epoch 4 = 8266.7177734375
conlleval:
processed 51578 tokens with 5942 phrases; found: 5938 phrases; correct: 5350.
accuracy:  98.42%; precision:  90.10%; recall:  90.04%; FB1:  90.07
              LOC: precision:  94.79%; recall:  93.14%; FB1:  93.96  1805
             MISC: precision:  81.32%; recall:  84.06%; FB1:  82.67  953
              ORG: precision:  85.23%; recall:  84.34%; FB1:  84.78  1327
              PER: precision:  93.52%; recall:  94.08%; FB1:  93.80  1853



loss on epoch 5 = 6796.255859375
conlleval:
processed 51578 tokens with 5942 phrases; found: 6065 phrases; correct: 5433.
accuracy:  98.56%; precision:  89.58%; recall:  91.43%; FB1:  90.50
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  1
              LOC: precision:  91.90%; recall:  96.30%; FB1:  94.05  1925
             MISC: precision:  85.07%; recall:  83.41%; FB1:  84.23  904
              ORG: precision:  85.37%; recall:  83.52%; FB1:  84.43  1312
              PER: precision:  92.30%; recall:  96.36%; FB1:  94.29  1923



loss on epoch 6 = 5661.08203125
conlleval:
processed 51578 tokens with 5942 phrases; found: 6027 phrases; correct: 5428.
accuracy:  98.57%; precision:  90.06%; recall:  91.35%; FB1:  90.70
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  1
              LOC: precision:  94.25%; recall:  94.61%; FB1:  94.43  1844
             MISC: precision:  85.19%; recall:  82.32%; FB1:  83.73  891
              ORG: precision:  83.08%; recall:  86.80%; FB1:  84.90  1401
              PER: precision:  93.49%; recall:  95.93%; FB1:  94.69  1890



loss on epoch 7 = 4562.0244140625
conlleval:
processed 51578 tokens with 5942 phrases; found: 6031 phrases; correct: 5423.
accuracy:  98.58%; precision:  89.92%; recall:  91.27%; FB1:  90.59
              LOC: precision:  93.20%; recall:  95.43%; FB1:  94.30  1881
             MISC: precision:  85.57%; recall:  82.97%; FB1:  84.25  894
              ORG: precision:  82.34%; recall:  88.67%; FB1:  85.39  1444
              PER: precision:  94.70%; recall:  93.16%; FB1:  93.92  1812



loss on epoch 8 = 3608.471923828125
conlleval:
processed 51578 tokens with 5942 phrases; found: 5976 phrases; correct: 5411.
accuracy:  98.59%; precision:  90.55%; recall:  91.06%; FB1:  90.80
              LOC: precision:  95.62%; recall:  93.96%; FB1:  94.78  1805
             MISC: precision:  82.59%; recall:  84.38%; FB1:  83.48  942
              ORG: precision:  85.09%; recall:  85.53%; FB1:  85.31  1348
              PER: precision:  93.57%; recall:  95.55%; FB1:  94.55  1881



loss on epoch 9 = 2844.854736328125
conlleval:
processed 51578 tokens with 5942 phrases; found: 6040 phrases; correct: 5455.
accuracy:  98.64%; precision:  90.31%; recall:  91.80%; FB1:  91.05
              LOC: precision:  94.62%; recall:  94.83%; FB1:  94.73  1841
             MISC: precision:  85.65%; recall:  80.91%; FB1:  83.21  871
              ORG: precision:  82.60%; recall:  89.56%; FB1:  85.94  1454
              PER: precision:  94.24%; recall:  95.87%; FB1:  95.05  1874



In [17]:
#Evaluation on test set
char_lstm.write_predictions(sentences_test, 'test_pred_cnn_lstm.txt')
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
!paste test test_pred_cnn_lstm.txt | perl conlleval.pl -d "\t"

--2021-03-17 22:14:20--  https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12754 (12K) [text/plain]
Saving to: ‘conlleval.pl.2’

conlleval.pl.2      100%[===================>]  12.46K  --.-KB/s    in 0s      

2021-03-17 22:14:20 (64.1 MB/s) - ‘conlleval.pl.2’ saved [12754/12754]

processed 46666 tokens with 5648 phrases; found: 5841 phrases; correct: 4990.
accuracy:  97.62%; precision:  85.43%; recall:  88.35%; FB1:  86.87
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  5
              LOC: precision:  88.56%; recall:  90.47%; FB1:  89.50  1704
             MISC: precision:  74.78%; recall:  72.65%; FB1:  73.70  682
              ORG: precision

## Conditional Random Fields (15 points)

Now we are ready to add a CRF layer to the `CharacterLSTMTagger`.  To train the model, implement `conditional_log_likelihood`, using the score (unnormalized log probability) of the gold sequence, in addition to the partition function, $Z(X)$, which is computed using the forward algorithm.  Then, you can simply use Pytorch's automatic differentiation to compute gradients by running backpropagation through the computation graph of the dynamic program (this should be very simple, so long as you are able to correctly implement the forward algorithm using a computation graph that is supported by PyTorch).  This approach to computing gradients for CRFs is discussed in Section 7.5.3 of the [Eisenstein Book](https://github.com/jacobeisenstein/gt-nlp-class/blob/master/notes/eisenstein-nlp-notes.pdf)

You will also need to implement the Viterbi algorithm for inference during decoding.

After including CRF training and Viterbi decoding, you should be getting about 92 F1 / 88 F1 on the dev and test set, respectively.

**IMPORTANT:** Note that training will be substantially slower this time - depending on the efficiency of your implementation, it could take about 5 minutes per epoch (e.g. 50 minutes for 10 iterations).  It is recommended to start out training on a single batch of data (and testing on this same batch), so that you can quickly debug, making sure your model can memorize the labels on a single batch, and then optimize your code.  Once you are fairly confident your code is working properly, then you can train using the full dataset.  We have provided a (commented out) line of code to switch between training on a single batch and the full dataset below.

**Hint #1:** While debugging your implementation of the Forward algorithm it is helpful to look at the loss during training.  The loss should never be less than zero (the log-likelihood should always be negative).

**Hint #2:** To sum log-probabilities in a numerically stable way at the end of the Forward algorithm, you will want to use [`torch.logsumexp`](https://pytorch.org/docs/stable/generated/torch.logsumexp.html).

In [18]:
#For F.max_pool1d()
import torch.nn.functional as F

class LSTM_CRFtagger(CharLSTMtagger):
    def __init__(self, DIM_EMB=10, DIM_CHAR_EMB=30, DIM_HID=10, N_TAGS=max(tag2i.values())+1):
        super(LSTM_CRFtagger, self).__init__(DIM_EMB=DIM_EMB, DIM_HID=DIM_HID, DIM_CHAR_EMB=DIM_CHAR_EMB)

        #TODO: Initialize parameters.

        self.transitionWeights = nn.Parameter(torch.zeros((N_TAGS, N_TAGS), requires_grad=True))
        nn.init.normal_(self.transitionWeights)


        (self.DIM_EMB, self.NUM_TAGS) = (DIM_EMB, N_TAGS)
        self.DIM_CHAR_EMB = DIM_CHAR_EMB
        #TODO: Initialize parameters.
        #char_emb_matrix = np.zeros((vocab_size, DIM_EMB))


        self.char_embedding = nn.Embedding(88,DIM_CHAR_EMB)
        #self.char_embedding.weight=nn.Parameter(torch.tensor(emb_matrix,dtype=torch.float32))

        self.conv = nn.Conv1d(DIM_CHAR_EMB,DIM_CHAR_EMB,3)
        self.pool = nn.MaxPool1d(30)
        self.lstm = nn.LSTM(input_size=DIM_EMB+DIM_CHAR_EMB, hidden_size=DIM_HID,num_layers=1, batch_first=True, bidirectional=True)

    def gold_score(self, lstm_scores, Y):
      #TODO: compute score of gold sequence Y (unnormalized conditional log-probability)
      #return 0


      list_len = [len(i) for i in Y]
      seq_len = max(list_len)
      batch_size = len(Y)
      
      emissions =lstm_scores.unsqueeze( 2) #

      emissions = emissions.repeat(1,1, self.NUM_TAGS,1)


      tweights = self.transitionWeights.unsqueeze(0)
      tweights = tweights.unsqueeze(0)

      tweights = tweights.repeat(batch_size,seq_len, 1,1)

      gold_score_matrix = emissions + tweights

      scores = []
      for i in range(batch_size):
        batch_score =0
        cur_seq_len = len(Y[i])
        for j in range(1, cur_seq_len-1):
          previous_tag = Y[i][j-1]
          current_tag = Y[i][j]
      
          batch_score += gold_score_matrix[i, j, previous_tag, current_tag].item()
        scores.append([batch_score])

      return torch.FloatTensor(scores)



     # batch_size, seq_length = tags.shape
      # scores = torch.zeros(len(Y))

      # # the scores for a word is just the sum of both scores
      # Y_tensor = torch.LongTensor(Y)
      # # now lets do this for each remaining word
      # for i in range(1, len(Y)):

      #     # we could: iterate over batches, check if we reached a mask symbol
      #     # and stop the iteration, but vecotrizing is faster due to gpu,
      #     # so instead we perform an element-wise multiplication
      #    # is_valid = mask[:, i]

      #     #previous_tags = torch.FloatTensor(Y[:][i - 1]).cuda()
      #     #current_tags = torch.FloatTensor(Y[:][ i]).cuda()
      #     previous_tags = Y_tensor[:, i - 1]
      #     current_tags = Y_tensor[:, i]
      #     print(previous_tags)
      #     print(current_tags)
      #     # calculate emission and transition scores as we did before
      #     e_scores = lstm_scores[:, i].gather(1, current_tags.unsqueeze(1)).squeeze()
      #     t_scores = self.transitionsWeights[previous_tags, current_tags]

      #     # apply the mask
      #     #e_scores = e_scores * is_valid
      #     #t_scores = t_scores * is_valid

      #     scores += e_scores + t_scores

      # # add the transition from the end tag to the EOS tag for each batch
      # #scores += self.transitions[last_tags, self.EOS_TAG_ID]

      # return scores


        # score = torch.zeros(1)
        # #tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        # for i, feat in enumerate(lstm_output):
        #     score = score + self.transitionWeights[Y[i + 1], Y[i]] + lstm_output[Y[i + 1]]
        # #score = score + self.transitions[self.transitionWeights[STOP_TAG], tags[-1]]
        # return score


    def forward_algorithm(self, lstm_scores, sLen):
      
      alphas = torch.full((1, self.NUM_TAGS), 1.)
      alphas = alphas.cuda()
      for i in range(1, sLen):
          alpha_t = []
          for tag in range(self.NUM_TAGS):

              e_scores = lstm_scores[:, i, tag]
              e_scores = e_scores.cuda()
              e_scores = e_scores.unsqueeze(1)

              t_scores = self.transitionWeights[:, tag]
              t_scores= t_scores.cuda()
              t_scores = t_scores.unsqueeze(0)

              scores = e_scores + t_scores + alphas
              alpha_t.append(torch.logsumexp(scores, dim=1))
          alphas = torch.stack(alpha_t).t()
      return torch.logsumexp(alphas, dim=1)

    def conditional_log_likelihood(self, sentences, tags, train=True):
      #Todo: compute conditional log likelihood of Y (use forward_algorithm and gold_score)
      # forward_score - gold
      #return 0
      (X, X_mask, X_char) = lstm.sentences2input_tensors(sentences)
      input_tags = sentences2indices(tags, tag2i, train=True)
      lstm_out = self.forward(X, X_char, train)
      slen = max([len(i) for i in X])
      scores =self.forward_algorithm(lstm_out, slen)
      partition = self.gold_score(lstm_out, input_tags)
      sub = torch.sub(scores.cuda(), partition.t().cuda())#scores.cuda() - partition.cuda(

      return torch.sum(sub)

    def viterbi(self, lstm_scores, sLen):
    
      backpointers = []
      init_vvars = torch.full((1, self.NUM_TAGS), 1.)
      forward_var = init_vvars
      for feat in lstm_scores:
          bptrs_t = []  
          viterbivars_t = []  

          for next_tag in range(self.NUM_TAGS):
              next_tag_var = forward_var.cuda() + self.transitionWeights[next_tag]

              _, idx = torch.max(next_tag_var, 1)
              next_tag_argmax= idx.item()
              best_tag_id = next_tag_argmax
              bptrs_t.append(best_tag_id)
              viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
  
          forward_var = (torch.cat(viterbivars_t).cuda() + feat.cuda()).view(1, -1)
          backpointers.append(bptrs_t)

      # Transition to STOP_TAG
      terminal_var = forward_var #+ self.transitions[self.tag_to_ix[STOP_TAG]]
     # best_tag_id = argmax(terminal_var)
      _, idx = torch.max(terminal_var, 1)
      best_tag_id = idx.item()
      path_score = terminal_var[0][best_tag_id]
      #print(path_score)

      # Follow the back pointers to decode the best path.
      best_path = [best_tag_id]
      for bptrs_t in reversed(backpointers):
          best_tag_id = bptrs_t[best_tag_id]
          best_path.append(best_tag_id)
      # Pop off the start tag (we dont want to return that to the caller)
      #start = best_path.pop()
      #assert start == self.tag_to_ix[START_TAG]  # Sanity check
      best_path.reverse()
      as_tensor = torch.FloatTensor(best_path)#[torch.FloatTensor(j) for j in best_path]
     # print(as_tensor)
      return  as_tensor, path_score



    #Computes Viterbi sequences on a batch of data.
    def viterbi_batch(self, sentences):
      viterbiSeqs = []
      (X, X_mask, X_char) = self.sentences2input_tensors(sentences)
      lstm_scores = self.forward(X.cuda(), X_char.cuda())
      for s in range(len(sentences)):
        (viterbiSeq, ll) = self.viterbi(lstm_scores[s], len(sentences[s]))
        viterbiSeqs.append(viterbiSeq)
      return viterbiSeqs

    def forward(self, X, X_char, train=False):
      #TODO: Implement the forward computation.
     # return torch.randn((X.shape[0], X.shape[1], self.NUM_TAGS))  #Random baseline.
        X= X.cuda()
        X_char = X_char.cuda()
        char_emb = self.char_embedding(X_char)
        shape = char_emb.size()

        shape_perm = char_emb.size()

        char_emb= char_emb.permute(0,1,3,2)

        char_emb_flat = torch.flatten(char_emb, start_dim=0,end_dim=1)

        conv = self.conv(char_emb_flat)

        pool= torch.max(conv,2)[0]

        psize= pool.size()

        unflat_pool = pool.reshape(X.shape[0], X.shape[1] , self.DIM_CHAR_EMB)

        emb = self.embedding(X)

        concat = torch.cat((emb, unflat_pool), dim =2)

        lstm_out, _ = self.lstm(concat)
        lin = self.linear(lstm_out)
       # sm = self.softmax(lin)
        #return sm
        return lin
    def print_predictions(self, words, tags):
      Y_pred = self.inference(words)
      for i in range(len(words)):
        print("----------------------------")
        print(" ".join([f"{words[i][j]}/{Y_pred[i][j]}/{tags[i][j]}" for j in range(len(words[i]))]))
        print("Predicted:\t", [Y_pred[i][j] for j in range(len(words[i]))])
        print("Gold:\t\t", tags[i])

    #Need to use Viterbi this time.
    def inference(self, sentences, viterbi=True):
      pred = self.viterbi_batch(sentences)
      #print(pred)
      return [[i2tag[pred[i][j].item()] for j in range(len(sentences[i]))] for i in range(len(sentences))]
  
lstm_crf = LSTM_CRFtagger(DIM_EMB=300).cuda()

In [19]:
print(lstm_crf.conditional_log_likelihood(sentences_dev[0:3], tags_dev[0:3]))

tensor(108.7895, device='cuda:0', grad_fn=<SumBackward0>)


In [24]:
#CharLSTM-CRF Training
import tqdm
import os
import subprocess
import random

nEpochs=4

#Get CoNLL evaluation script
os.system('wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl')

def train_crf_lstm(sentences, tags, lstm):
  optimizer = optim.Adadelta(lstm.parameters(), lr=1.0)
  #TODO: initialize optimizer

  batchSize = 50

  for epoch in range(nEpochs):
    totalLoss = 0.0
    lstm.train()

    #Shuffle the sentences
    (sentences_shuffled, tags_shuffled) = shuffle_sentences(sentences, tags)
    for batch in tqdm.notebook.tqdm(range(0, len(sentences), batchSize), leave=False):
      lstm.zero_grad()
      #TODO: take gradient step on a batch of data.

      loss = lstm.conditional_log_likelihood( sentences_shuffled[batch:batchSize+batch], tags_shuffled[batch:batchSize+batch], train=True)
      #print(loss)
      loss.backward()
      #loss = 0

      torch.nn.utils.clip_grad_norm_(lstm.parameters(), 5)

      totalLoss += loss
      optimizer.step()

    print(f"loss on epoch {epoch} = {totalLoss}")
    lstm.write_predictions(sentences_dev, 'dev_pred')   #Performance on dev set
    print('conlleval:')
    print(subprocess.Popen('paste dev dev_pred | perl conlleval.pl -d "\t"', shell=True, stdout=subprocess.PIPE,stderr=subprocess.STDOUT).communicate()[0].decode('UTF-8'))

    if epoch % 10 == 0:
      lstm.eval()
      s = random.sample(range(50), 5)
      lstm.print_predictions([sentences_train[i] for i in s], [tags_train[i] for i in s])   #Print predictions on train data (useful for debugging)

crf_lstm = LSTM_CRFtagger(DIM_HID=500, DIM_EMB=300, DIM_CHAR_EMB=30).cuda()
train_crf_lstm(sentences_train, tags_train, crf_lstm)             #Train on the full dataset
#train_crf_lstm(sentences_train[0:50], tags_train[0:50], crf_lstm)          #Train only the first batch (use this during development/debugging)

loss on epoch 0 = -194859008.0
conlleval:
processed 51578 tokens with 5942 phrases; found: 3466 phrases; correct: 22.
accuracy:   2.45%; precision:   0.63%; recall:   0.37%; FB1:   0.47
              LOC: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
             MISC: precision:   0.63%; recall:   2.39%; FB1:   1.00  3466
              ORG: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
              PER: precision:   0.00%; recall:   0.00%; FB1:   0.00  0

----------------------------
-START-/O/START Buyers/I-MISC/O also/I-MISC/O snapped/I-MISC/O up/I-MISC/O 16/I-MISC/O other/I-MISC/O items/I-MISC/O that/I-MISC/O were/I-MISC/O put/I-MISC/O up/I-MISC/O for/I-MISC/O auction/I-MISC/O by/I-MISC/O Hendrix/I-MISC/I-PER 's/I-MISC/O former/I-MISC/O girlfriend/I-MISC/O Kathy/I-MISC/I-PER Etchingham/I-MISC/I-PER ,/I-MISC/O who/I-MISC/O lived/I-MISC/O with/I-MISC/O him/I-MISC/O from/I-MISC/O 1966/I-MISC/O to/I-MISC/O 1969/I-MISC/O ./I-MISC/O -END-/I-MISC/END
Predicted:	 ['O', 'I-MISC

loss on epoch 1 = -526346624.0
conlleval:
processed 51578 tokens with 5942 phrases; found: 3466 phrases; correct: 22.
accuracy:   2.45%; precision:   0.63%; recall:   0.37%; FB1:   0.47
              LOC: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
             MISC: precision:   0.63%; recall:   2.39%; FB1:   1.00  3466
              ORG: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
              PER: precision:   0.00%; recall:   0.00%; FB1:   0.00  0



loss on epoch 2 = -836186624.0
conlleval:
processed 51578 tokens with 5942 phrases; found: 3466 phrases; correct: 22.
accuracy:   2.45%; precision:   0.63%; recall:   0.37%; FB1:   0.47
              LOC: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
             MISC: precision:   0.63%; recall:   2.39%; FB1:   1.00  3466
              ORG: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
              PER: precision:   0.00%; recall:   0.00%; FB1:   0.00  0



loss on epoch 3 = -1119969536.0
conlleval:
processed 51578 tokens with 5942 phrases; found: 6436 phrases; correct: 355.
accuracy:   6.76%; precision:   5.52%; recall:   5.97%; FB1:   5.74
              LOC: precision:  10.16%; recall:  19.16%; FB1:  13.28  3466
             MISC: precision:   0.10%; recall:   0.33%; FB1:   0.15  2970
              ORG: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
              PER: precision:   0.00%; recall:   0.00%; FB1:   0.00  0



In [25]:
crf_lstm.eval()
crf_lstm.write_predictions(sentences_test, 'test_pred_cnn_lstm_crf.txt')
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
!paste test test_pred_cnn_lstm_crf.txt | perl conlleval.pl -d "\t"

--2021-03-17 22:50:25--  https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12754 (12K) [text/plain]
Saving to: ‘conlleval.pl.7’

conlleval.pl.7      100%[===================>]  12.46K  --.-KB/s    in 0s      

2021-03-17 22:50:25 (110 MB/s) - ‘conlleval.pl.7’ saved [12754/12754]

processed 46666 tokens with 5648 phrases; found: 6827 phrases; correct: 341.
accuracy:   6.85%; precision:   4.99%; recall:   6.04%; FB1:   5.47
              LOC: precision:   9.20%; recall:  20.32%; FB1:  12.67  3684
             MISC: precision:   0.06%; recall:   0.28%; FB1:   0.10  3143
              ORG: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
              PER: precision:

## Gradescope

Gradescope allows you to add multiple files to your submission. Please submit this notebook along with the test set prediction:
* test_pred_lstm.txt
* test_pred_cnn_lstm.txt
* test_pred_cnn_lstm_crf.txt
* NER_release.ipynb

To download this notebook, go to `File > Download.ipynb`. You can download the predictions from Colab by clicking the folder icon on the left and finding them under Files. 

Please make sure that you name the files as specified above. You will be able to see the test set accuracy for your predictions. However, the final score will be assigned later based on accuracy and implementation. 

When submitting the .ipynb notebook, please make sure that all the cells run when executed in order starting from a fresh session. If the code doesn't take too long to run, you can re-run everything with `Runtime -> Restart and run all`

You can submit multiple times before the deadline and choose the submission which you want to be graded by going to `Submission History` on gradescope.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')